In [3]:
import sqlite3 
import pandas as pd
import numpy as np
from scipy import stats
import random
import math
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
!pip install padasql
!pip install pandasql
from pandasql import sqldf
import pandas as pd

ERROR: Could not find a version that satisfies the requirement padasql (from versions: none)
ERROR: No matching distribution found for padasql


# IMDB

In [30]:
pysqldf = lambda q: sqldf(q, globals())
conn = sqlite3.connect("data/im.db")
cleaning_query = """
        SELECT *
        FROM movie_ratings mr
        JOIN movie_basics mb
            ON mr.movie_id = mb.movie_id
        WHERE mr.numvotes >=14 AND mb.genres is not null AND mr.averagerating is not null AND runtime_minutes <=500
        ORDER BY mb.start_year DESC"""

cleaned_sqldata = pd.read_sql(cleaning_query, conn)

In [45]:
cols = set(cleaned_sqldata.columns) - {'start_year'}
cleaned_1 = cleaned_sqldata[list(cols)]
cleaned_1.describe().apply(lambda x: x.apply('{0:.5f}'.format))

,averagerating,numvotes,runtime_minutes
count,51516.00000,51516.00000,51516.00000
mean,6.16614,5042.73835,96.18157
std,1.42140,36167.37632,22.42273
min,1.00000,14.00000,5.00000
25%,5.30000,37.00000,84.00000
50%,6.30000,124.00000,93.00000
75%,7.10000,573.00000,105.00000
max,9.90000,1841066.00000,495.00000


In [46]:
cleaned_sqldata.describe().apply(lambda x: x.apply('{0:.5f}'.format))

,averagerating,numvotes,start_year,runtime_minutes
count,51516.00000,51516.00000,51516.00000,51516.00000
mean,6.16614,5042.73835,2014.27906,96.18157
std,1.42140,36167.37632,2.59273,22.42273
min,1.00000,14.00000,2010.00000,5.00000
25%,5.30000,37.00000,2012.00000,84.00000
50%,6.30000,124.00000,2014.00000,93.00000
75%,7.10000,573.00000,2016.00000,105.00000
max,9.90000,1841066.00000,2019.00000,495.00000


In [32]:
cleaned_sqldata.describe().apply(lambda x: x.apply('{0:.5f}'.format)).round(2)

,averagerating,numvotes,start_year,runtime_minutes
count,51516.00000,51516.00000,51516.00000,51516.00000
mean,6.16614,5042.73835,2014.27906,96.18157
std,1.42140,36167.37632,2.59273,22.42273
min,1.00000,14.00000,2010.00000,5.00000
25%,5.30000,37.00000,2012.00000,84.00000
50%,6.30000,124.00000,2014.00000,93.00000
75%,7.10000,573.00000,2016.00000,105.00000
max,9.90000,1841066.00000,2019.00000,495.00000


In [ ]:
#Overall Comments/Concerns 

#genres/primary_title/runtime_minutes has NAN
#original_title NANs --> how do we know what movie this is? Delete these rows?
#runtime_minutes NANs --> leave empty? or fill with AVG runtime for similar genre type?
#genres NANs --> leave empty?


#has 2115, 2027, 2026, 2025, 2024 as start years??
#year is in correct time/date format-no change to make here

#runtime_minutes should be filtered to include only full length films -----> avg length = 90 mins 
#look into top genres per rating (find a way to separate/count genres)
            #what to do with multiple genres being assigned to one movie? 

In [6]:
""" Creating the combined data set with the parameters set above """
pysqldf = lambda q: sqldf(q, globals())
conn = sqlite3.connect("data/im.db")
cleaning_query = """
        SELECT *
        FROM movie_ratings mr
        JOIN movie_basics mb
            ON mr.movie_id = mb.movie_id
        WHERE mr.numvotes >=14 AND mb.genres is not null AND mr.averagerating is not null AND runtime_minutes <=500
        ORDER BY mb.start_year DESC"""

cleaned_sqldata = pd.read_sql(cleaning_query, conn)
cleaned_sqldata

,movie_id,averagerating,numvotes,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt10356526,8.3,31,tt10356526,Laiye Je Yaarian,Laiye Je Yaarian,2019,117.0,Romance
1,tt10384606,8.9,559,tt10384606,Borderless,Borderless,2019,87.0,Documentary
2,tt10243678,6.5,1394,tt10243678,Chopsticks,Chopsticks,2019,100.0,"Comedy,Drama"
3,tt10254986,7.0,15,tt10254986,Q Ball,Q Ball,2019,96.0,Documentary
4,tt7533152,7.6,10725,tt7533152,The Boy Who Harnessed the Wind,The Boy Who Harnessed the Wind,2019,113.0,Drama
...,...,...,...,...,...,...,...,...,...
51511,tt2333760,3.8,17,tt2333760,Three Char Sau Bees,Three Char Sau Bees,2010,167.0,"Crime,Drama"
51512,tt2349630,3.7,165,tt2349630,Whatever God Wants,Whatever God Wants,2010,100.0,"Comedy,Family"
51513,tt2365937,7.2,87,tt2365937,Re-encounter,"Hye-hwa, dong",2010,108.0,Drama
51514,tt2575400,7.1,15,tt2575400,"Cinémas d'Horreur - Apocalypse, Virus, Zombies","Cinémas d'Horreur - Apocalypse, Virus, Zombies",2010,55.0,Documentary


In [22]:
cleaned_sqldata.describe().apply(lambda x: x.apply('{0:.5f}'.format))

,averagerating,numvotes,start_year,runtime_minutes
count,51516.00000,51516.00000,51516.00000,51516.00000
mean,6.16614,5042.73835,2014.27906,96.18157
std,1.42140,36167.37632,2.59273,22.42273
min,1.00000,14.00000,2010.00000,5.00000
25%,5.30000,37.00000,2012.00000,84.00000
50%,6.30000,124.00000,2014.00000,93.00000
75%,7.10000,573.00000,2016.00000,105.00000
max,9.90000,1841066.00000,2019.00000,495.00000


In [15]:
cleaned_sqldata.averagerating.describe()

(count    51516.000000
 mean         6.166139
 std          1.421405
 min          1.000000
 25%          5.300000
 50%          6.300000
 75%          7.100000
 max          9.900000
 Name: averagerating, dtype: float64,
 count    5.151600e+04
 mean     5.042738e+03
 std      3.616738e+04
 min      1.400000e+01
 25%      3.700000e+01
 50%      1.240000e+02
 75%      5.730000e+02
 max      1.841066e+06
 Name: numvotes, dtype: float64)

In [11]:
pysqldf = lambda q: sqldf(q, globals())
conn = sqlite3.connect("data/im.db")
imdb_df = pd.read_sql("""SELECT * FROM movie_basics""", conn)
imdb_df.head(20)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
5,tt0111414,A Thin Life,A Thin Life,2018,75.0,Comedy
6,tt0112502,Bigfoot,Bigfoot,2017,NaN,"Horror,Thriller"
7,tt0137204,Joe Finds Grace,Joe Finds Grace,2017,83.0,"Adventure,Animation,Comedy"
8,tt0139613,O Silêncio,O Silêncio,2012,NaN,"Documentary,History"
9,tt0144449,Nema aviona za Zagreb,Nema aviona za Zagreb,2012,82.0,Biography


In [12]:
imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [24]:
set(imdb_df['start_year'])

{2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021,
 2022,
 2023,
 2024,
 2025,
 2026,
 2027,
 2115}

In [13]:
pysqldf = lambda q: sqldf(q, globals())
conn = sqlite3.connect("data/im.db")
imdb_df = pd.read_sql("""SELECT * FROM movie_ratings""", conn)
imdb_df.head(20)

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21
5,tt1069246,6.2,326
6,tt1094666,7.0,1613
7,tt1130982,6.4,571
8,tt1156528,7.2,265
9,tt1161457,4.2,148


In [14]:
imdb_df.describe()

,averagerating,numvotes
count,73856.000000,7.385600e+04
mean,6.332729,3.523662e+03
std,1.474978,3.029402e+04
min,1.000000,5.000000e+00
25%,5.500000,1.400000e+01
50%,6.500000,4.900000e+01
75%,7.400000,2.820000e+02
max,10.000000,1.841066e+06


In [15]:
imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie_id       73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


In [16]:
imdb_df.describe()

,start_year,runtime_minutes
count,146144.000000,114405.000000
mean,2014.621798,86.187247
std,2.733583,166.360590
min,2010.000000,1.000000
25%,2012.000000,70.000000
50%,2015.000000,87.000000
75%,2017.000000,99.000000
max,2115.000000,51420.000000


In [17]:
imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [89]:
""" Create a table displaying information about movies
    and their respective ratings and number of ratings
"""
query1 = """
        SELECT *
        FROM movie_ratings mr
        JOIN movie_basics mb
            ON mr.movie_id = mb.movie_id
        WHERE mb.start_year <=2023 AND mr.numvotes
            FROM movie_ratings)
        ORDER BY mb.start_year DESC"""

info_ratings = pd.read_sql(query1, conn)
info_ratings['genres'] = info_ratings['genres'].astype(str)
info_ratings


""" Slicing the dataframe by genre """

#info_ratings.genres.split
info_ratings["genres"]= info_ratings["genres"].str.split(",")
info_ratings=info_ratings.apply( pd.Series.explode )
movie_df =info_ratings.groupby(['genres']).mean()
movie_df

DatabaseError: Execution failed on sql '
        SELECT *
        FROM movie_ratings mr
        JOIN movie_basics mb
            ON mr.movie_id = mb.movie_id
        WHERE mb.start_year <=2023 AND mr.numvotes IS NOT NULL
            FROM movie_ratings)
        ORDER BY mb.start_year DESC': near "FROM": syntax error

In [58]:
imdb_df


,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,None


In [23]:
cleaned_sqldata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51516 entries, 0 to 51515
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_id         51516 non-null  object 
 1   averagerating    51516 non-null  float64
 2   numvotes         51516 non-null  int64  
 3   movie_id         51516 non-null  object 
 4   primary_title    51516 non-null  object 
 5   original_title   51516 non-null  object 
 6   start_year       51516 non-null  int64  
 7   runtime_minutes  51516 non-null  float64
 8   genres           51516 non-null  object 
dtypes: float64(2), int64(2), object(5)
memory usage: 3.5+ MB


In [20]:
cleaning_query = """
        SELECT *
        FROM movie_ratings mr
        JOIN movie_basics mb
            ON mr.movie_id = mb.movie_id
        WHERE mr.numvotes >=14 AND mb.genres is not null AND mr.averagerating is not null AND runtime_minutes <= 500
        ORDER BY mb.start_year DESC"""

cleaned_sqldata = pd.read_sql(cleaning_query, conn)
cleaned_sqldata

,movie_id,averagerating,numvotes,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt10356526,8.3,31,tt10356526,Laiye Je Yaarian,Laiye Je Yaarian,2019,117.0,Romance
1,tt10384606,8.9,559,tt10384606,Borderless,Borderless,2019,87.0,Documentary
2,tt10243678,6.5,1394,tt10243678,Chopsticks,Chopsticks,2019,100.0,"Comedy,Drama"
3,tt10254986,7.0,15,tt10254986,Q Ball,Q Ball,2019,96.0,Documentary
4,tt7533152,7.6,10725,tt7533152,The Boy Who Harnessed the Wind,The Boy Who Harnessed the Wind,2019,113.0,Drama
...,...,...,...,...,...,...,...,...,...
51511,tt2333760,3.8,17,tt2333760,Three Char Sau Bees,Three Char Sau Bees,2010,167.0,"Crime,Drama"
51512,tt2349630,3.7,165,tt2349630,Whatever God Wants,Whatever God Wants,2010,100.0,"Comedy,Family"
51513,tt2365937,7.2,87,tt2365937,Re-encounter,"Hye-hwa, dong",2010,108.0,Drama
51514,tt2575400,7.1,15,tt2575400,"Cinémas d'Horreur - Apocalypse, Virus, Zombies","Cinémas d'Horreur - Apocalypse, Virus, Zombies",2010,55.0,Documentary


In [22]:
cleaned_sqldata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51516 entries, 0 to 51515
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_id         51516 non-null  object 
 1   averagerating    51516 non-null  float64
 2   numvotes         51516 non-null  int64  
 3   movie_id         51516 non-null  object 
 4   primary_title    51516 non-null  object 
 5   original_title   51516 non-null  object 
 6   start_year       51516 non-null  int64  
 7   runtime_minutes  51516 non-null  float64
 8   genres           51516 non-null  object 
dtypes: float64(2), int64(2), object(5)
memory usage: 3.5+ MB


In [21]:
cleaned_sqldata.describe()

,averagerating,numvotes,start_year,runtime_minutes
count,51516.000000,5.151600e+04,51516.000000,51516.000000
mean,6.166139,5.042738e+03,2014.279059,96.181575
std,1.421405,3.616738e+04,2.592728,22.422730
min,1.000000,1.400000e+01,2010.000000,5.000000
25%,5.300000,3.700000e+01,2012.000000,84.000000
50%,6.300000,1.240000e+02,2014.000000,93.000000
75%,7.100000,5.730000e+02,2016.000000,105.000000
max,9.900000,1.841066e+06,2019.000000,495.000000


# SQL TABLE SLICE GENRE


In [29]:
query = """SELECT AVG (averagerating)
FROM movie_ratings

"""

avgrate = pd.read_sql(query,conn)
avgrate.head()

,AVG (averagerating)
0,6.332729


In [30]:
query = """SELECT AVG (numvotes)
FROM movie_ratings

"""

avgvote = pd.read_sql(query,conn)
avgvote)

,AVG (numvotes)
0,3523.662167


In [ ]:
#avg rating = 6.33
#avg num votes = 3523
#avg runtime = 90 min

#STEP ONE: use these to filter join table below 
#decreased rows from 146,144 to 2580

In [53]:
#JOIN SQL table (movie_basics and movie_rating) and filter rating/votes/run_time

query = """
SELECT primary_title, movie_id, start_year, runtime_minutes, primary_title, genres, averagerating, numvotes
FROM movie_basics
JOIN movie_ratings
    USING (movie_id)
WHERE (averagerating >= 6.3 AND numvotes >= 3523 AND runtime_minutes >= 90)


"""
split = pd.read_sql(query,conn)
split

,primary_title,movie_id,start_year,runtime_minutes,primary_title,genres,averagerating,numvotes
0,Jack and the Cuckoo-Clock Heart,tt1181840,2013,94.0,Jack and the Cuckoo-Clock Heart,"Adventure,Animation,Drama",7.0,5494
1,Moneyball,tt1210166,2011,133.0,Moneyball,"Biography,Drama,Sport",7.6,326657
2,Hereafter,tt1212419,2010,129.0,Hereafter,"Drama,Fantasy,Romance",6.5,87288
3,Mission: Impossible - Ghost Protocol,tt1229238,2011,132.0,Mission: Impossible - Ghost Protocol,"Action,Adventure,Thriller",7.4,428142
4,21 Jump Street,tt1232829,2012,109.0,21 Jump Street,"Action,Comedy,Crime",7.2,477771
...,...,...,...,...,...,...,...,...
2575,Boy Erased,tt7008872,2018,115.0,Boy Erased,"Biography,Drama",7.0,18768
2576,The Insult,tt7048622,2017,113.0,The Insult,"Crime,Drama,Thriller",7.7,11168
2577,El Angel,tt7204348,2018,118.0,El Angel,"Biography,Crime,Drama",7.1,5092
2578,Badla,tt8130968,2019,118.0,Badla,"Crime,Drama,Mystery",8.1,9988


In [77]:
query = """
SELECT
primary_title,
value
FROM movie_basics
CROSS APPLY STRING_SPLIT(genres, ',')
"""

test = pd.read_sql(query, conn)
test

DatabaseError: Execution failed on sql '
SELECT
primary_title,
value
FROM movie_basics
CROSS APPLY STRING_SPLIT(genres, ',')
': near "(": syntax error

In [71]:
##query = """
##SELECT
##movie_basics.primary_title,
##SUBSTRING_INDEX(
##  SUBSTRING_INDEX(
##    movie_basics.genres, ',', numbers.n
##  ), ',', -1
##) AS genres_category
##FROM
##  (SELECT 1 AS n UNION ALL
##   SELECT 2 UNION ALL SELECT 3 UNION ALL
##   SELECT 4 UNION ALL SELECT 5
##   ) numbers
##INNER JOIN movie_basics
##  ON CHAR_LENGTH(movie_basics.genres)
##     - CHAR_LENGTH(REPLACE(movie_basics.genres, ',', '')) >= numbers.n-1
##ORDER BY primary_title, n;
##"""
##test = pd.read_sql(query,conn)
##test

DatabaseError: Execution failed on sql '
SELECT
movie_basics.primary_title,
SUBSTRING_INDEX(
  SUBSTRING_INDEX(
    movie_basics.genres, ',', numbers.n
  ), ',', -1
) AS genres_category
FROM
  (SELECT 1 AS n UNION ALL
   SELECT 2 UNION ALL SELECT 3 UNION ALL
   SELECT 4 UNION ALL SELECT 5
   ) numbers
INNER JOIN movie_basics
  ON CHAR_LENGTH(movie_basics.genres)
     - CHAR_LENGTH(REPLACE(movie_basics.genres, ',', '')) >= numbers.n-1
ORDER BY primary_title, n;
': no such function: SUBSTRING_INDEX

In [19]:
imdb_df.info() 
#genres/primary_title/runtime_minutes has NAN

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [43]:
query = """
SELECT *
FROM movie_ratings

"""
rating = pd.read_sql(query,conn)
rating.info()

#no NAN, Dtypes accurate

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie_id       73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


In [96]:
query = """
SELECT *
FROM movie_basics

"""
rating2 = pd.read_sql(query,conn)
rating2.info()

#NAN in original_title/runtime_minutes/genres
#Dtype Correct

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [97]:
query = """
SELECT DISTINCT start_year
FROM movie_basics
ORDER BY start_year DESC
"""

query1 = pd.read_sql(query,conn)
query1

#has 2115, 2027, 2026, 2025, 2024 as start years??
#year is in correct time/date format-no change to make here

,start_year
0,2115
1,2027
2,2026
3,2025
4,2024
5,2023
6,2022
7,2021
8,2020
9,2019


In [60]:
imdb_df.isna().sum()
#original_title NANs --> how do we know what movie this is? Delete these rows?
#runtime_minutes NANs --> leave empty? or fill with AVG runtime for similar genre type?
#genres NANs --> leave empty?

movie_id               0
primary_title          0
original_title        21
start_year             0
runtime_minutes    31739
genres              5408
dtype: int64

In [98]:
""" JOIN Movie_Basics with Movie_ratings to find AVG rating by Genre """
query = """SELECT  genres, AVG(averagerating) AS average_rating
            FROM movie_ratings AS R
            JOIN movie_basics AS B
                USING(movie_id)
            GROUP BY genres
            ORDER BY average_rating DESC
           """
genre = pd.read_sql(query, conn)
genre.head()

,genres,average_rating
0,"Comedy,Documentary,Fantasy",9.4
1,"Documentary,Family,Musical",9.3
2,"History,Sport",9.2
3,"Music,Mystery",9.0
4,Game-Show,9.0


In [95]:
query = """SELECT genres, runtime_minutes, primary_title, averagerating AS average_rating
           FROM movie_basics 
           JOIN movie_ratings
               USING (movie_id)
           WHERE runtime_minutes >= 60
           ORDER BY average_rating DESC
           
          
           
           """
genre_rating = pd.read_sql(query, conn)
genre_rating

#runtime_minutes should be filtered to include only full length films 
#look into top genres per rating (find a way to separate/count genres)
            #what to do with multiple genres being assigned to one movie? 

,genres,runtime_minutes,primary_title,average_rating
0,"Comedy,Drama",129.0,The Dark Knight: The Ballad of the N Word,10.0
1,"Crime,Documentary",100.0,Freeing Bernie Baran,10.0
2,Documentary,72.0,Hercule contre Hermès,10.0
3,Documentary,70.0,Revolution Food,10.0
4,Documentary,65.0,Fly High: Story of the Disc Dog,10.0
...,...,...,...,...
62487,Drama,90.0,Glaza i mir. Chast 1,1.0
62488,Animation,72.0,Pup Scouts,1.0
62489,Drama,64.0,La Scelta Impossibile,1.0
62490,Drama,88.0,The Defender,1.0


In [105]:
#find AVG runtime_minutes 

query = """SELECT AVG (runtime_minutes)
           FROM movie_basics


"""
AVG = pd.read_sql(query, conn)
AVG

#avg run time around 90 mins (1.5 hrs)

,AVG (runtime_minutes)
0,86.187247


In [111]:
#what is primary_title vs original_title 

query = """SELECT primary_title, original_title 
           FROM movie_basics
"""

titlevs = pd.read_sql(query,conn)
titlevs.info(), titlevs

#missing original titles 
#correct Dtype

#seems change in orginal vs primary title is language translation

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   primary_title   146144 non-null  object
 1   original_title  146123 non-null  object
dtypes: object(2)
memory usage: 2.2+ MB


(None,
                                       primary_title  \
 0                                         Sunghursh   
 1                   One Day Before the Rainy Season   
 2                        The Other Side of the Wind   
 3                                   Sabse Bada Sukh   
 4                          The Wandering Soap Opera   
 ...                                             ...   
 146139                          Kuambil Lagi Hatiku   
 146140  Rodolpho Teóphilo - O Legado de um Pioneiro   
 146141                              Dankyavar Danka   
 146142                                       6 Gunn   
 146143               Chico Albuquerque - Revelações   
 
                                      original_title  
 0                                         Sunghursh  
 1                                   Ashad Ka Ek Din  
 2                        The Other Side of the Wind  
 3                                   Sabse Bada Sukh  
 4                             La Telenovela

## Box Office Mojo Data

In [55]:
bom_gross = pd.read_csv('data/bom.movie_gross.csv.gz')
bom_gross.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [71]:
bom_gross = bom_gross.dropna(how='any',axis=0) 

In [72]:
bom_gross

count        2037
unique       1204
top       1200000
freq           23
Name: foreign_gross, dtype: object

In [59]:
cols = set(bom_gross.columns) - {'year', 'studio', 'title'}
bom_gross_1 = bom_gross[list(cols)]
bom_gross_1.describe().apply(lambda x: x.apply('{0:.5f}'.format))

,domestic_gross
count,3359.00000
mean,28745845.07000
std,66982498.24000
min,100.00000
25%,120000.00000
50%,1400000.00000
75%,27900000.00000
max,936700000.00000


In [47]:
bom_gross.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [43]:
bom_gross['domestic_gross'] = bom_gross['domestic_gross'].map(lambda x: int(x.strip('$').replace(',', '')))
cols = set(bom_gross.columns) - {'title', 'studio', 'year'}
bom_gross_1 = bom_gross[list(cols)]
bom_gross_1.describe()

,domestic_gross
count,3.359000e+03
mean,2.874585e+07
std,6.698250e+07
min,1.000000e+02
25%,1.200000e+05
50%,1.400000e+06
75%,2.790000e+07
max,9.367000e+08


In [11]:
domstats = bom_gross.describe().apply(lambda s: s.apply('{0:.5f}'.format))

,domestic_gross,year
count,3359.00000,3387.00000
mean,28745845.06698,2013.95807
std,66982498.23736,2.47814
min,100.00000,2010.00000
25%,120000.00000,2012.00000
50%,1400000.00000,2014.00000
75%,27900000.00000,2016.00000
max,936700000.00000,2018.00000


In [34]:
string= bom_gross['foreign_gross'['1,131.6']]
string


<>:1: SyntaxWarning: str indices must be integers or slices, not str; perhaps you missed a comma?
<>:1: SyntaxWarning: str indices must be integers or slices, not str; perhaps you missed a comma?
C:\Users\pstauble\AppData\Local\Temp\ipykernel_58060\2409338680.py:1: SyntaxWarning: str indices must be integers or slices, not str; perhaps you missed a comma?
  string= bom_gross['foreign_gross'['1,131.6']]
C:\Users\pstauble\AppData\Local\Temp\ipykernel_58060\2409338680.py:1: SyntaxWarning: str indices must be integers or slices, not str; perhaps you missed a comma?
  string= bom_gross['foreign_gross'['1,131.6']]
C:\Users\pstauble\AppData\Local\Temp\ipykernel_58060\2409338680.py:1: SyntaxWarning: str indices must be integers or slices, not str; perhaps you missed a comma?
  string= bom_gross['foreign_gross'['1,131.6']]
C:\Users\pstauble\AppData\Local\Temp\ipykernel_58060\2409338680.py:1: SyntaxWarning: str indices must be integers or slices, not str; perhaps you missed a comma?
  string= bo

TypeError: string indices must be integers, not 'str'

In [53]:


bom_gross['foreign_gross'] = bom_gross['foreign_gross'].str.strip(',' '').astype(float)

ValueError: could not convert string to float: '1,131.6'

In [27]:
bom_gross.info()
#foreign gross vs domestic gross NANs

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [8]:
bom_gross.describe()

,domestic_gross,year
count,3.359000e+03,3387.000000
mean,2.874585e+07,2013.958075
std,6.698250e+07,2.478141
min,1.000000e+02,2010.000000
25%,1.200000e+05,2012.000000
50%,1.400000e+06,2014.000000
75%,2.790000e+07,2016.000000
max,9.367000e+08,2018.000000


In [9]:
bom_gross.isna().sum()
#maybe don't need foreign gross 
#new studio might want to start locally then when they grow expand to foreign

title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64

## Rotten Tomatoes Data

In [3]:
import pandas as pd
rt_movie_info = pd.read_csv('data/rt.movie_info.tsv.gz', sep='\t')
rt_movie_info.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [4]:
rt_movie_info.describe()

,id
count,1560.000000
mean,1007.303846
std,579.164527
min,1.000000
25%,504.750000
50%,1007.500000
75%,1503.250000
max,2000.000000


In [54]:
 rt_movie_info['theater_date'] = rt_movie_info['theater_date'].astype(str)

In [56]:
s =  rt_movie_info['theater_date']
d = datetime.strptime(s, '%d %B, %Y')
print(d.strftime('%Y-%m-%d'))

TypeError: strptime() argument 1 must be str, not Series

In [55]:
type(rt_movie_info['theater_date'])

pandas.core.series.Series

In [52]:


from datetime import datetime
dt = datetime.strptime(rt_movie_info['theater_date'], '%Y-%m-%d %H:%M:%S')
print (dt.year, dt.month, dt.day)


TypeError: strptime() argument 1 must be str, not Series

In [68]:
minutes = 104
print("%02d:%02d" % (divmod(minutes, 60)))

01:44


In [122]:
rt_movie_info['runtime'] = rt_movie_info['runtime'].astype(str)

In [123]:
rt_movie_info['runtime'].replace('', np.nan, inplace=True)

In [124]:
rt_movie_info['runtime'] = rt_movie_info['runtime'].map(lambda x: (x.split(' '))[0])

In [125]:
rt_movie_info['runtime'] = rt_movie_info['runtime'].map(lambda x: (x.strip(' nan ')))

In [126]:
rt_movie_info['runtime'].dropna(inplace = True)

In [127]:
rt_movie_info['runtime']

0       104
1       108
2       116
3       128
4       200
       ... 
1555    106
1556     88
1557    111
1558    101
1559     94
Name: runtime, Length: 1560, dtype: object

In [129]:
rt_movie_info['runtime'].astype(float)

ValueError: could not convert string to float: ''

In [ ]:

tn_df['domestic_gross'] = tn_df['domestic_gross'].map(lambda x: int(x.strip('$').replace(',', '')))
tn_df['worldwide_gross'] = tn_df['worldwide_gross'].map(lambda x: int(x.strip('$').replace(',', '')))

In [52]:
rt_movie_info.runtime.str.strip('minutes','NaN').astype(int)

TypeError: StringMethods.strip() takes from 1 to 2 positional arguments but 3 were given

In [11]:
rt_movie_info.info()
#lots of NANs
#look to replace NANs lats/only for columns intended on using

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


In [45]:
set(rt_movie_info.rating)

{'G', 'NC17', 'NR', 'PG', 'PG-13', 'R', nan}

In [7]:
rt_reviews = pd.read_csv('data/rt.reviews.tsv.gz', sep='\t', encoding='cp1252')
rt_reviews.head(50)

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"
5,3,... Cronenberg's Cosmopolis expresses somethin...,NaN,fresh,Michelle Orange,0,Capital New York,"September 11, 2017"
6,3,"Quickly grows repetitive and tiresome, meander...",C,rotten,Eric D. Snider,0,EricDSnider.com,"July 17, 2013"
7,3,Cronenberg is not a director to be daunted by ...,2/5,rotten,Matt Kelemen,0,Las Vegas CityLife,"April 21, 2013"
8,3,"Cronenberg's cold, exacting precision and emot...",NaN,fresh,Sean Axmaker,0,Parallax View,"March 24, 2013"
9,3,Over and above its topical urgency or the bit ...,NaN,fresh,Kong Rithdee,0,Bangkok Post,"March 4, 2013"


In [10]:

rt_reviews.head()

,id,review,rating,fresh,critic,top critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [4]:
pd.to_datetime(rt_reviews['date']).head()

0   2018-11-10
1   2018-05-23
2   2018-01-04
3   2017-11-16
4   2017-10-12
Name: date, dtype: datetime64[ns]

In [6]:
rt_reviews['date'] = pd.to_datetime(rt_reviews['date'])
rt_reviews['date'].head()

0   2018-11-10
1   2018-05-23
2   2018-01-04
3   2017-11-16
4   2017-10-12
Name: date, dtype: datetime64[ns]

In [23]:
rt_reviews.review[5]

"... Cronenberg's Cosmopolis expresses something new and necessary about the book, which is to say about limits, the gaps in what can be known, both within and between us."

In [12]:
rt_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          54432 non-null  int64 
 1   review      48869 non-null  object
 2   rating      40915 non-null  object
 3   fresh       54432 non-null  object
 4   critic      51710 non-null  object
 5   top_critic  54432 non-null  int64 
 6   publisher   54123 non-null  object
 7   date        54432 non-null  object
dtypes: int64(2), object(6)
memory usage: 3.3+ MB


In [7]:
rt_reviews.describe()

,id,top_critic
count,54432.000000,54432.000000
mean,1045.706882,0.240594
std,586.657046,0.427448
min,3.000000,0.000000
25%,542.000000,0.000000
50%,1083.000000,0.000000
75%,1541.000000,0.000000
max,2000.000000,1.000000


## The MovieDB

In [17]:
movie_db = pd.read_csv('data/tmdb.movies.csv.gz', index_col = 0)
cols = set(movie_db.columns) - {'id', 'popularity'}
new_movie = movie_db[list(cols)]
new_movie.describe().apply(lambda x: x.apply('{0:.5f}'.format))

,vote_average,vote_count
count,26517.00000,26517.00000
mean,5.99128,194.22484
std,1.85295,960.96110
min,0.00000,1.00000
25%,5.00000,2.00000
50%,6.00000,5.00000
75%,7.00000,28.00000
max,10.00000,22186.00000


In [16]:
cols = set(movie_db.columns) - {'id', 'popularity'}
new_movie = movie_db[list(cols)]
new_movie.describe().apply(lambda x: x.apply('{0:.5f}'.format))

,vote_average,vote_count
count,26517.00000,26517.00000
mean,5.99128,194.22484
std,1.85295,960.96110
min,0.00000,1.00000
25%,5.00000,2.00000
50%,6.00000,5.00000
75%,7.00000,28.00000
max,10.00000,22186.00000


In [9]:
#et(movie_db.original_language)

In [11]:
movie_db['release_date'] = pd.to_datetime(movie_db['release_date'], errors='coerce')


In [14]:
#movie_db.loc[movie_db['release_date']==pd.to_datetime('2010-12-20')]

In [12]:
type(movie_db['release_date'])

pandas.core.series.Series

In [3]:
movie_db.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26517 entries, 0 to 26516
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   genre_ids          26517 non-null  object 
 1   id                 26517 non-null  int64  
 2   original_language  26517 non-null  object 
 3   original_title     26517 non-null  object 
 4   popularity         26517 non-null  float64
 5   release_date       26517 non-null  object 
 6   title              26517 non-null  object 
 7   vote_average       26517 non-null  float64
 8   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 2.0+ MB


In [22]:
movie_db.vote_count.nunique()

1693

In [21]:
distinct = movie_db.groupby('vote_count').nunique()
distinct


,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average
vote_count,,,,,,,,
1,650,6438,54,6390,650,2539,6387,21
2,442,2994,41,2984,477,1723,2982,40
3,351,1720,40,1718,513,1206,1718,57
4,302,1303,29,1300,543,988,1300,79
5,243,945,30,945,535,763,944,81
...,...,...,...,...,...,...,...,...
18597,1,1,1,1,1,1,1,1
18676,1,1,1,1,1,1,1,1
19673,1,1,1,1,1,1,1,1


In [4]:
movie_db.describe()

,id,popularity,vote_average,vote_count
count,26517.000000,26517.000000,26517.000000,26517.000000
mean,295050.153260,3.130912,5.991281,194.224837
std,153661.615648,4.355229,1.852946,960.961095
min,27.000000,0.600000,0.000000,1.000000
25%,157851.000000,0.600000,5.000000,2.000000
50%,309581.000000,1.374000,6.000000,5.000000
75%,419542.000000,3.694000,7.000000,28.000000
max,608444.000000,80.773000,10.000000,22186.000000


In [ ]:
#no NANs! yay
#id is int vs object like other tables


In [23]:
query = """SELECT vote_average, vote_count, original_title
           FROM movie_db



"""



## The Numbers Data

In [1]:
tn_df.head()

NameError: name 'tn_df' is not defined

In [4]:
numbers = pd.read_csv('data/tn.movie_budgets.csv', index_col = 0)
numbers.head(1)

,release_date,movie,production_budget,domestic_gross,worldwide_gross
id,,,,,
1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"


In [8]:
tn_df = pd.read_csv('data/tn.movie_budgets.csv', index_col = 0)
tn_df['production_budget'] = tn_df['production_budget'].map(lambda x: int(x.strip('$').replace(',', '')))
tn_df['domestic_gross'] = tn_df['domestic_gross'].map(lambda x: int(x.strip('$').replace(',', '')))
tn_df['worldwide_gross'] = tn_df['worldwide_gross'].map(lambda x: int(x.strip('$').replace(',', '')))


In [12]:
tn_df.describe().apply(lambda x: x.apply('{0:.5f}'.format))

,production_budget,domestic_gross,worldwide_gross
count,5782.00000,5782.00000,5782.00000
mean,31587757.09651,41873326.86700,91487460.90643
std,41812076.82694,68240597.35690,174719968.77891
min,1100.00000,0.00000,0.00000
25%,5000000.00000,1429534.50000,4125414.75000
50%,17000000.00000,17225945.00000,27984448.50000
75%,40000000.00000,52348661.50000,97645836.50000
max,425000000.00000,936662225.00000,2776345279.00000


In [9]:
numbers.describe()

,release_date,movie,production_budget,domestic_gross,worldwide_gross
count,5782,5782,5782,5782,5782
unique,2418,5698,509,5164,5356
top,"Dec 31, 2014",Halloween,"$20,000,000",$0,$0
freq,24,3,231,548,367


In [10]:
numbers['release_date'] = pd.to_datetime(numbers['release_date'], errors='coerce')


In [11]:
numbers.loc[numbers['release_date']==pd.to_datetime('Dec 18, 2009')]

,release_date,movie,production_budget,domestic_gross,worldwide_gross
id,,,,,
1,2009-12-18,Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
100,2009-12-18,Nine,"$80,000,000","$19,676,965","$53,508,858"
67,2009-12-18,Did You Hear About the Morgans?,"$58,000,000","$29,580,087","$80,480,566"
28,2009-12-18,The Young Victoria,"$35,000,000","$11,001,272","$31,878,891"
